In [183]:
import pandas as pd

In [221]:
df = pd.read_excel('home_work.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [185]:
data.Rating.value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Preprocessing

In [227]:
!pip install stop_words
!pip install pymorphy2
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [230]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [232]:
df['new_content'] = df.Content.apply(preprocess_text)

In [233]:
df.head(3)

,Rating,Content,Date,new_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично


In [234]:
train = df[:int(len(df) * 0.7 - 1)]
test = df[int(len(df) * 0.7 - 1):]

Data Filtration

In [191]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(" ".join(train.new_content))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [192]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [193]:
from nltk.probability import FreqDist
max_words = 200
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [194]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'супер',
 'телефон']

In [195]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [196]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [197]:
max_len = 40
x_train = np.asarray([text_to_sequence(text, max_len) for text in train["new_content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test["new_content"]], dtype=np.int32)

In [198]:
y_train = np.array(train.Rating.tolist())
y_test = np.array(test.Rating.tolist())

In [199]:
y_train

array([5, 4, 5, ..., 3, 5, 5])

# Keras model

In [200]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  

In [201]:
num_classes = 5 
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [202]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40, 128)           25600     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 38, 128)           49280     
_________________________________________________________________
activation_12 (Activation)   (None, 38, 128)           0         
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_13 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                

In [203]:
train["Rating"] = train["Rating"].apply(lambda x: x -1 )
test["Rating"] = test["Rating"].apply(lambda x: x -1 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [204]:
y_train = keras.utils.to_categorical(train["Rating"])
y_val = keras.utils.to_categorical(test["Rating"])

In [205]:
y_train[0]

array([0., 0., 0., 0., 1.], dtype=float32)

In [206]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [207]:
y_train.shape

(14460, 5)

In [208]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
batch_size = 512
epochs = 20
print_batch_n = 100

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, ])#early_stopping])

Epoch 1/20
26/26 [==============================] - 5s 162ms/step - loss: 1.4270 - accuracy: 0.5475 - val_loss: 1.2656 - val_accuracy: 0.6923
Epoch 2/20
26/26 [==============================] - 4s 153ms/step - loss: 1.1676 - accuracy: 0.7330 - val_loss: 1.1441 - val_accuracy: 0.6923
Epoch 3/20
26/26 [==============================] - 4s 154ms/step - loss: 1.0023 - accuracy: 0.7337 - val_loss: 0.9040 - val_accuracy: 0.7130
Epoch 4/20
26/26 [==============================] - 4s 154ms/step - loss: 0.7436 - accuracy: 0.7482 - val_loss: 0.7930 - val_accuracy: 0.7303
Epoch 5/20
26/26 [==============================] - 4s 152ms/step - loss: 0.6850 - accuracy: 0.7659 - val_loss: 0.7713 - val_accuracy: 0.7303
Epoch 6/20
26/26 [==============================] - 4s 153ms/step - loss: 0.6639 - accuracy: 0.7715 - val_loss: 0.7597 - val_accuracy: 0.7324
Epoch 7/20
26/26 [==============================] - 4s 153ms/step - loss: 0.6518 - accuracy: 0.7732 - val_loss: 0.7555 - val_accuracy: 0.7344
Epoch 

In [209]:
score = model.evaluate(x_test, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 40ms/step - loss: 0.7679 - accuracy: 0.7409


Test score: 0.767925500869751
Test accuracy: 0.7409259676933289


Инициализировать tf.keras.layers.Embedding предобученными векторами

In [210]:
import os
import pathlib

data_path = keras.utils.get_file(
    "182.zip",
    "http://vectors.nlpl.eu/repository/20/182.zip",
    untar=True,
)

637624320/637613799 [==============================] - 25s 0us/step


In [211]:
!ls /root/.keras/datasets/

182.zip  182.zip.tar.gz  195.zip


In [212]:
os.rename('/root/.keras/datasets/182.zip.tar.gz', '/root/.keras/datasets/182.zip')
!unzip /root/.keras/datasets/182.zip 

Archive:  /root/.keras/datasets/182.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               A

  inflating: README                  


In [214]:
embeddings_index = {}
f = open('model.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 248979 word vectors.


In [256]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

texts = df.new_content.tolist()
tokenizer = Tokenizer(nb_words=200)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=max_len)



/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 13485 unique tokens.


In [288]:
labels = np.array((df['Rating'].tolist()))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Shape of data tensor: (20659, 40)
Shape of label tensor: (20659,)


In [289]:
labels = labels - 1

In [290]:
VALIDATION_SPLIT = 0.3
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [291]:
y_val

array([4, 4, 4, ..., 4, 4, 2])

In [298]:
y_train.shape

(14462,)

In [307]:
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

In [308]:
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [309]:
EMBEDDING_DIM = 128
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [310]:
max_words

200

In [314]:
num_classes = 5 
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, weights=[embedding_matrix], input_length=128, trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [315]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 128, 128)          1726208   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 126, 128)          49280     
_________________________________________________________________
activation_33 (Activation)   (None, 126, 128)          0         
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                1290      
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 5)               

In [316]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
batch_size = 512
epochs = 20
print_batch_n = 100

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, ])#early_stopping])

Epoch 1/20
26/26 [==============================] - 4s 123ms/step - loss: 1.5971 - accuracy: 0.6833 - val_loss: 1.5827 - val_accuracy: 0.7194
Epoch 2/20
26/26 [==============================] - 3s 113ms/step - loss: 1.5714 - accuracy: 0.7053 - val_loss: 1.5566 - val_accuracy: 0.7194
Epoch 3/20
26/26 [==============================] - 3s 112ms/step - loss: 1.5463 - accuracy: 0.7053 - val_loss: 1.5311 - val_accuracy: 0.7194
Epoch 4/20
26/26 [==============================] - 3s 113ms/step - loss: 1.5220 - accuracy: 0.7053 - val_loss: 1.5064 - val_accuracy: 0.7194
Epoch 5/20
26/26 [==============================] - 3s 112ms/step - loss: 1.4983 - accuracy: 0.7053 - val_loss: 1.4824 - val_accuracy: 0.7194
Epoch 6/20
26/26 [==============================] - 3s 112ms/step - loss: 1.4752 - accuracy: 0.7053 - val_loss: 1.4590 - val_accuracy: 0.7194
Epoch 7/20
26/26 [==============================] - 3s 111ms/step - loss: 1.4529 - accuracy: 0.7053 - val_loss: 1.4364 - val_accuracy: 0.7194
Epoch 